# kaggle mnist by keras

with accuracy  0.98943 with nb_epoch = 12

original code from https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py


## 1. Read kaggle mnist


In [2]:
import numpy as np
import pandas as pd

# create the training & test sets, skipping the header row with [1:]
dataset = pd.read_csv("../../tmp/train.csv")
target = dataset[[0]].values.ravel()
train = dataset.iloc[:,1:].values
test = pd.read_csv("../../tmp/test.csv").values

X_test = test.reshape((len(test),1,28,28))
X_train = train.reshape((len(train),1,28,28))
y_train = target

## 2. CNN model


In [4]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.utils import np_utils


batch_size = 128
nb_classes = 10
nb_epoch = 2

# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
nb_pool = 2
# convolution kernel size
nb_conv = 3


X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)

model = Sequential()

model.add(Convolution2D(nb_filters, nb_conv, nb_conv,
                        border_mode='valid',
                        input_shape=(1, img_rows, img_cols)))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters, nb_conv, nb_conv))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adadelta')

X_train shape: (42000, 1, 28, 28)
42000 train samples
28000 test samples


## 3. Train model


In [6]:
X_valid = X_train[0:2000]#40000 train
Y_valid = Y_train[0:2000]#2000 validation
model.fit(X_train[2000:], Y_train[2000:], batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=(X_valid, Y_valid))

Train on 40000 samples, validate on 2000 samples
Epoch 1/2
40000/40000 [==============================] - 119s - loss: 2.3018 - val_loss: 2.3037
Epoch 2/2
40000/40000 [==============================] - 119s - loss: 2.3015 - val_loss: 2.3038


# 4. predict and save to csv


In [ ]:
pred = model.predict(X_test)
pred = np.argmax(pred, 1)
np.savetxt('submission_cnn_keras_kaggle.csv', np.c_[range(1,len(test)+1),pred], delimiter=',', header = 'ImageId,Label', comments = '', fmt='%d')


## 5. Save trained model


In [7]:
def save_keras_model(model, path):
    with open(path + '.json', 'w') as f:
        f.write(model.to_json())
    model.save_weights(path+'.h5', overwrite=True)

save_keras_model(model, 'model_saved')


## 6. Load saved model

In [11]:
model.load_weights('model_saved.h5')